In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [4]:
# Filter out for SampleIDs in Pairs
# Load up the mapping file
df_mapping = pd.read_csv("../data/SampleID_map.txt", sep='\t', index_col=0)
df_mapping.head()

,UserName,StudyDayNo,StudyDate,Gender,Age,Weight,Height,BMI,Supplement,Waist.Circumference,...,fecal.time,BMI.1,Weight.Change,Plate,SampleOrder,SampleType,Timing,Activity.Factor,Medications,Dietary.Supp
#SampleID,,,,,,,,,,,,,,,,,,,,,
Blank.A02.,Blank_A02,17,2/16/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,Blank,NaN,NaN,NaN,NaN
Blank.A03.,Blank_A03,17,2/16/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,Blank,NaN,NaN,NaN,NaN
Blank.A04.,Blank_A04,17,2/16/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,Blank,NaN,NaN,NaN,NaN
Blank.A07.,Blank_A07,17,2/16/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,NaN,Blank,NaN,NaN,NaN,NaN
Blank.B12.,Blank_B12,17,2/16/17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,Blank,NaN,NaN,NaN,NaN


In [31]:
# Load up the sample names in data
df_samples = pd.read_csv("../data/taxonomy_clr_s.txt", sep='\t', index_col=0)
df_samples.head()
sample_names = set(df_samples.columns)

In [32]:
# Load up the sample names in food data
df_food_samples = pd.read_csv("../data/dhydrt.txt", sep='\t', index_col=0)
df_food_samples.head()
sample_names = sample_names.intersection(set(df_food_samples.columns))

In [33]:
def yield_subsample_difference_dfs(df_mapping, sample_names):
    # Retrieve a vector sample numbers, in order of date
    for group, df in df_mapping.groupby('UserName'):
        if 'Blank' not in group:
            df = df.sort_values('StudyDayNo')
            # Verify after filter there are more than 1 sample
            df = df[[_ in sample_names for _ in df.index]]
            if df.shape[0] > 1:
                diff = np.diff(df['StudyDayNo']) == 1
                # Double the first value for indexing
                diff = np.insert(diff, 0, diff[0], axis=0)
                df = df[diff]
                # Verify after filter there are more than 1 sample
                if df.shape[0] > 1:
                    yield df.copy()

In [34]:
subsample_df = pd.concat(yield_subsample_difference_dfs(df_mapping, sample_names))

In [35]:
subsample_df.shape

(405, 23)

In [36]:
df_mapping.shape

(643, 23)

In [37]:
# Write to file
subsample_df.to_csv("../results/filtered.sampleids.txt", sep="\t")